In [1]:
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os, sys, email
import gensim
from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from string import punctuation
import timeit
from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
start = timeit.default_timer()
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
from smart_open import smart_open
from random import shuffle

pd.set_option('display.max_colwidth',1000)

In [77]:
# Load dataset
data = pd.read_csv('processed_resumes_work_ADDED_JOB_TITLES.csv')

In [78]:
# Removed jobs without description
data = data[~data.descript.isnull()]

# Convert null job title into blank
data.loc[data.role.isnull()] = ""

In [79]:
# Create dataset for training doc2vec
data['train_val'] = data.role + " " + data.descript.str.strip()

In [80]:
data['train_val'].count()

1229127

In [81]:
data['train_val'].nunique()

1214940

In [10]:
def clean_data(data):
    stop_words = set(stopwords.words('english'))
    lmtzr = WordNetLemmatizer()

    cleaned_list = []
    
    for row in data:
        row = re.sub(r'[^a-zA-Z ]+', ' ', row.lower())
        row = ' '.join([lmtzr.lemmatize(word) for word in row.split() if word not in stop_words])
        cleaned_list.append(row)    
    
    return cleaned_list    

In [25]:
original_data[328412]

'application architect .TOKYO, JAPAN\xa0.ING L fe I su ce Comp y (Jp)Lm ted p ov des fe su ce p oducts d se v ces\xa0.fo E ow e s. ING L Po t (INGLP)sfudmet system povdgv ous\xa0.se v ces of the ING fe su ce to Age t, R, Custome d ING emp oyees. Th s INGLP s\xa0.etypot the web whe e ING c p omote ew p oducts d se v ces, teg te\xa0.d ffe e t web ppctosfop ocess gdhdg custome tsctos, docume t\xa0.p oduct gudeesdp ov de st uct o s d educ to cotetfoRsd ge ts..'

In [24]:
cleaned_data.index("application architect tokyo japan ing l fe su ce comp jp lm ted p ov de fe su ce p oducts se v ce fo e ow e ing l po inglp sfudmet system povdgv ous se v ce ing fe su ce age r custome ing emp oyees th inglp etypot web whe e ing c p omote ew p oducts se v ce teg te ffe e web ppctosfop ocess gdhdg custome tsctos docume p oduct gudeesdp ov de st uct educ cotetforsd ge t")

328412

In [23]:
[s for s in cleaned_data if "application architect tokyo japan" in s]

['application architect tokyo japan ing l fe su ce comp jp lm ted p ov de fe su ce p oducts se v ce fo e ow e ing l po inglp sfudmet system povdgv ous se v ce ing fe su ce age r custome ing emp oyees th inglp etypot web whe e ing c p omote ew p oducts se v ce teg te ffe e web ppctosfop ocess gdhdg custome tsctos docume p oduct gudeesdp ov de st uct educ cotetforsd ge t']

In [11]:
original_data = data.train_val.unique()

In [20]:
# cleaned_data = clean_data(original_data)

In [5]:
# with open('cleaned_data', 'wb') as fp:
#     pickle.dump(cleaned_data, fp)

In [6]:
cleaned_data = pickle.load(open('cleaned_data', 'rb'))

In [71]:
cleaned_engineer_dat = [s for s in cleaned_data if "engineer" in s or "sales" in s]

In [ ]:
len(traindocs)

In [72]:
# Currently testing model process on 100,000 records
# 95k for training set, 5k for test set
traindocs = []
testdocs = []
for index, line in enumerate(cleaned_engineer_dat):
    words = gensim.utils.to_unicode(line).split()
    split = ['train', 'test'][index//1200000]
    
    if split == 'train':
        traindocs.append(TaggedDocument(words, [index]))
    else:
        testdocs.append(TaggedDocument(words, [index]))

print('%d train, %d test' % (len(traindocs), len(testdocs)))

191467 train, 0 test


In [470]:
with open('traindocs', 'wb') as fp:
    pickle.dump(traindocs, fp)

In [471]:
with open('testdocs', 'wb') as fp:
    pickle.dump(testdocs, fp)

In [73]:
model = gensim.models.Doc2Vec(traindocs, dm = 1, dbow_words = 1, 
                              negative = 5, alpha=0.025, size= 200, 
                              min_alpha=0.001, min_count=0, epochs=10
                             )
 
# model.train(traindocs, total_examples=175605, epochs=1)

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [74]:
model.most_similar('process')

/Users/kwheatley/anaconda/envs/python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('workflow', 0.6198469400405884),
 ('system', 0.5893020629882812),
 ('methodology', 0.5851994752883911),
 ('procedure', 0.5841983556747437),
 ('program', 0.5720818042755127),
 ('method', 0.5653647184371948),
 ('operation', 0.5644145607948303),
 ('project', 0.5632549524307251),
 ('activity', 0.5632162094116211),
 ('application', 0.5554951429367065)]

In [58]:
def find_most_similar(doc_id):
    if isinstance(doc_id, int):
        print(doc_id, 'seed_doc', ' '.join(traindocs[doc_id].words))
    else:
        print(doc_id)
    print()
        
    examples = model.docvecs.most_similar(doc_id)
    for example in examples:
        index = example[0]
        percent = example[1]
        text = ' '.join(traindocs[index].words)
        print(index, percent, text)
        print()

In [59]:
find_most_similar(85916)

85916 seed_doc sr business analyst sale engineer responsible product demonstration installation design sintecmedia software respond rfi rfp prospective current customer cable amp amp broadcast network mso mvpd write functional design sintecmedia software covering adsales right management program scheduling promotion finance provide product demonstration training site remotely area sintecmedia broadcast management software act project manager gap analysis phase implementation participate analysis provide marketing material powerpoint presentation one sheet etc provide input product roadmaps r amp amp

40151 0.6598896384239197 composer recording mixing engineer guitarist vocalist manager write arrange music organize lead rehearsal responsible recording production mixing recent project

12636 0.6573036909103394 electrical engineer provide electrical design custom control panel drawing electrical schematic field wiring panel wiring

9826 0.6550716161727905 system engineer provide ad exchan

In [60]:
# filevector = open('docvectors.txt', 'w')
vector_list = []
meta_list = []
# filemeta = open('docmeta.txt', 'w')
# edu_filemeta.write("%s\n" % 'title')

for index in range(len(traindocs)):
    cleaned_vectors = '\t'.join(str(vector) for vector in model.docvecs[index])
    if not cleaned_vectors or cleaned_vectors == '':
        print("issue with vector")
        print(index)
    vector_list.append(cleaned_vectors)
#     filevector.write("%s\n" % cleaned_vectors)
    cleaned_meta = (str(index) + ' ' + ' '.join(traindocs[index].words))
#     if index >= 94999:
#         print(index)
    if not cleaned_meta:
        print("issue with meta")
        print(index)
#     filemeta.write("%s\n" %cleaned_meta )
    meta_list.append(cleaned_meta)

In [70]:
# len(vector_list)
# len(meta_list)
filevector = open('docvectors.txt', 'w')
filemeta = open('docmeta.txt', 'w')
for row in vector_list:
    filevector.write("%s\n" % row)
for row in meta_list:
    filemeta.write("%s\n" % row)

In [286]:
# Let's look at the data more. Are the majority of these jobs entry level or higher level?
# We will base it on their graduation date.

# Load dataset
edu_data = pd.read_csv('processed_resumes_education.csv',header=None)

In [287]:
edu_data.columns = ['city','resume_id','container','degree','school','location','dates']
edu_data.head()


,city,resume_id,container,degree,school,location,dates
0,atlanta,0004d469fc497102,education-items,Bachelor of Science in Computer Science,Wichita State University,"Wichita, KS",1990 to 1994
1,atlanta,0006216bf673dc16,education-items,NaN,Youth Academy Soccer Coach,NaN,2018
2,atlanta,0006216bf673dc16,education-items,Bachelor of Arts in History,Georgia State University,"Atlanta, GA",August 2004
3,atlanta,0006216bf673dc16,education-items,NaN,Wake Forest University,NaN,1995 to 1997
4,atlanta,000cb5da9472be95,education-items,Bachelor of Science in Human Resources Management,Louisiana State University,NaN,NaN


In [305]:
edu_data[edu_data.degree == 'n economics']

,city,resume_id,container,degree,school,location,dates,degree_changed


In [357]:
edu_data['degree_changed'] = edu_data.degree.str.strip()
edu_data = edu_data[~edu_data.degree_changed.isnull()]
edu_data = edu_data[edu_data.degree_changed != '']

In [381]:
# Currently testing model process on 100,000 records
# 95k for training set, 5k for test set
edu_traindocs = []
edu_testdocs = []
for index, line in enumerate(edu_cleaned_data):
    words = gensim.utils.to_unicode(line).split()
    edu_traindocs.append(TaggedDocument(words, [index]))

In [ ]:
# After we get the clustering of documents, we can try to derive the topic
# https://stackoverflow.com/questions/46047506/how-to-find-most-similar-terms-words-of-a-document-in-doc2vec?rq=1